## Experiments with prompts

In [3]:
import requests
from openai import OpenAI
import os
import nltk

In [4]:
#get api keys from env file
google_api_key = os.environ.get("GOOGLE_API_KEY")
openai_api_key = os.environ.get("OPENAI_API_KEY")
if google_api_key is not None:
    print("Google key found!")
if openai_api_key is not None:
    print("OpenAI key found!")

Google key found!
OpenAI key found!


#### Functions

In [9]:
def get_nearby_restaurants(location, radius=1500, keyword='restaurant', num_results=5, tag=None, page=1):
    '''
    Returns the restaurants from the API call based on location, radius
    @
    '''
    base_url = 'https://maps.googleapis.com/maps/api/place/'
    
    # search for restaurants without specify the kind of restaurants
    if not tag :
        print("STARTING NO TAG SEARCH")
        params = {
            'location': location,
            # 'radius': radius,
            'keyword': keyword,
            'key': google_api_key,
            'rankby': 'distance'
        }
        response = requests.get(base_url+'nearbysearch/json', params=params)
    else:
        print("TAG SEARCH")
        query = tag.replace(',', ' or ')
        query = query + ' restaurant'

        print("query: ", query)

        params = {
            'location': location,
            'radius': radius,
            'query': query,
            'key': google_api_key,
        } 
        response = requests.get(base_url+'textsearch/json', params=params)

    results = response.json().get('results', [])

    #Basic pagination
    page_results = num_results*page
    previous_page = num_results*(page-1)

    # Limit the number of results to the top 'num_results'
    results = results[previous_page:page_results]
  
    return results


def get_place_details(place_id):
    '''
    Returns details on a place defined by the place_id, e.g. reviews
    '''
    base_url = 'https://maps.googleapis.com/maps/api/place/details/json'

    params = {
        'place_id': place_id,
        'key': google_api_key
    }

    response = requests.get(base_url, params=params)
    result = response.json().get('result', {})

    return result


def generate_summary(client, reviews, prompt):
    #Concatenate all reviews
    review_text = "\n".join(review.get('text', '') for review in reviews)

    #Define the prompt for GPT
    prompt = f"{prompt} \n{review_text}"

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt},
        ]
    )
    #Extract the generated summary from GPT's response
    summary = response.choices[0].message.content

    return summary

def fetch_review(place_id):
    cursor.execute('SELECT * FROM review_summaries WHERE id = ?', (place_id,))
    return cursor.fetchall()

def get_summary(place_id, prompt):
    # review_array = fetch_review(place_id)

    # if len(review_array) > 0:
    #     print("Found in database!")
    #     review = review_array[0]
    #     summary = review[1]
    # else:
    place_details = get_place_details(place_id)
    print("Summary for restaurant: ", place_details.get('name'))
    reviews = place_details.get('reviews', [])
    summary = generate_summary(OpenAI(), reviews, prompt)
    # cursor.execute('INSERT INTO review_summaries (id, review_summary) VALUES (?, ?)', (place_id, summary))
    # conn.commit()

    return summary

def get_all_summaries(restaurants, prompt):
    summaries = []
    for restaurant in restaurants:
        summaries.append(get_summary(restaurant['place_id'], prompt))

    return summaries

### Search

In [17]:
# LMU coordiantes
# lmu_coordinates = '48.148794,11.579980'
lmu_coordinates = '48.1450909,11.5743243'

# Set radius and keyword
# get_nearby_restaurants(location,tag, radius=500, keyword='restaurant', num_results=5)
restaurants = get_nearby_restaurants(lmu_coordinates, radius=500, keyword='restaurant', num_results=5, page=4)

# Print the names of the found restaurants
for restaurant in restaurants:
    #Print Restaurant name and average rating
    print("Restaurant name: ", restaurant.get('name'))
    print("Rating:  ", restaurant.get('rating'))
    
    print(restaurant)

STARTING NO TAG SEARCH
Restaurant name:  Türkenhof
Rating:   4.4
{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 48.1514056, 'lng': 11.5768359}, 'viewport': {'northeast': {'lat': 48.15276607989272, 'lng': 11.57814797989272}, 'southwest': {'lat': 48.15006642010727, 'lng': 11.57544832010728}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png', 'icon_background_color': '#FF9E67', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet', 'name': 'Türkenhof', 'opening_hours': {'open_now': False}, 'photos': [{'height': 3000, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/108505465088373482852">André Simões</a>'], 'photo_reference': 'AWU5eFhIuy7G5TpIBBUTG6iIjhhVwJpjnaaVIL7zKpVVbJ9dHSwnZp_pbQHh4oUcM0hQRNqTGpFf4OyoNzDuhztMBWlW9vlFZa7o9Pg35ZlW_G2E5YF7HeqBhynwlLrMPqZrvonh8nqkAv2WgM0SNeyAeHTNUuVdVUZ8lnvD-ZyK0QFix4CP', 'width': 4000}], 'place_id': 'ChIJn7g8pet1nkcRI9TkkPfwFVI', 'pl

In [8]:
# LMU coordiantes
# lmu_coordinates = '48.148794,11.579980'
lmu_coordinates = '48.1450909,11.5743243'
    
# Set radius and keyword
# get_nearby_restaurants(location,tag, radius=500, keyword='restaurant', num_results=5)
restaurants = get_nearby_restaurants(lmu_coordinates, radius=500, keyword='restaurant', num_results=20, tag='italian,indian,')

# Print the names of the found restaurants
for restaurant in restaurants:
    #Print Restaurant name and average rating
    print("Restaurant name: ", restaurant.get('name'))
    print("Rating:  ", restaurant.get('rating'))
    
    print(restaurant)

TAG SEARCH
query:  vegan and sushi restaurant
Restaurant name:  Little Tokyo
Rating:   4.1
{'business_status': 'OPERATIONAL', 'formatted_address': 'Steinheilstraße 21, 80333 München, Germany', 'geometry': {'location': {'lat': 48.1498137, 'lng': 11.5637016}, 'viewport': {'northeast': {'lat': 48.15121152989272, 'lng': 11.56491952989272}, 'southwest': {'lat': 48.14851187010727, 'lng': 11.56221987010728}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png', 'icon_background_color': '#FF9E67', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet', 'name': 'Little Tokyo', 'opening_hours': {'open_now': False}, 'photos': [{'height': 3024, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/108969179235530978806">A Google User</a>'], 'photo_reference': 'AWU5eFjOR_YajarPnsioHDrtUhwdqkNk1cTi5Bj-_N5BIPyZqT-A5YecYSgTITpU32imIQSzeScwQxOEy6B53bzULlTrQXsf5k5sD3GaWsQGAQL5RSA2RzMyz61ZSnycfcl0w3ZLroArRwcYkQ9A_

### Reviews Summary

##### Method 1: Overall Tendency

In [11]:
prompt = "I have 5 reviews for this restaurant. Please summarize it and give me the Important details about this restaurant. It is very important that you start the summary with the overall tendency of the review, e.g. mixed, negative, positive, very positive, etc."

get_all_summaries(restaurants, prompt)

Summary for restaurant:  Conti Restaurant
Summary for restaurant:  Wai's Kitchen
Summary for restaurant:  Oskar Maria Brasserie
Summary for restaurant:  Pietra Piccina
Summary for restaurant:  La Nonna


["Overall tendency of the review: Mixed\n\nImportant details about the restaurant:\n- The spargel-potatoe dish was described as mid, lacking outstanding flavors and being quite bland.\n- The kitchen closed before the reviewer could try their crème brulé.\n- The beer and one dish cost 36 euros, which the reviewer found pricey compared to a previous 3-course meal for 32 euros.\n- The reviewer did not like the terminals that prompt customers to tip and declined to tip.\n- The staff was generally courteous.\n- The reviewer felt like they were paying for the location rather than the food.\n- The reviewer did not endorse the restaurant and found the rating of 4.4 unfitting.\n- Another star was deducted for poor business practices.\n- One positive review mentioned that the food was tender and juicy, and the service was great.\n- Another positive review described the restaurant as having the best service and food in Munich.\n- Another positive review recommended trying the business lunch, whic

##### Method 2: No more than 4 Sentences

In [13]:
prompt = "Please give me a very short summary of the reviews for a restaurant. Do absolutely not use more than 4 sentences"

get_all_summaries(restaurants, prompt)

Summary for restaurant:  Conti Restaurant
Summary for restaurant:  Wai's Kitchen
Summary for restaurant:  Oskar Maria Brasserie
Summary for restaurant:  Pietra Piccina
Summary for restaurant:  La Nonna


["The restaurant received mixed reviews, with one reviewer finding the spargel-potato dish to be bland and the prices expensive. They also didn't like the tipping terminals. Another reviewer felt that they were paying for the location and had an underwhelming experience. However, there were positive reviews praising the tender meat, great service, and delicious food. One customer mentioned that it was their girlfriend's birthday, but the staff didn't acknowledge it. Overall, the reviews were mixed but leaned towards the negative side.",
 'The food is amazing, with fresh and authentic flavors. The service is extremely friendly and welcoming. The owners make you feel like family.',
 'Great coffee, delicious food, and live music at a reasonable volume. A cool spot to relax and spend time with friends.',
 'The restaurant has received rave reviews for its outstanding food, excellent service, and cozy atmosphere. Customers have praised the authentic Italian cuisine and the attentive waitstaf

##### Method 3:

In [16]:
prompt = "Please give me a summary of the reviews of this restaurant. I want one single summary for all 5 reviews. Please limit the summary to a maximum of 400 characters. It is extremely important that you limit the summary to a maximum of 400 characters."

get_all_summaries(restaurants, prompt)

Summary for restaurant:  Conti Restaurant
Summary for restaurant:  Wai's Kitchen
Summary for restaurant:  Oskar Maria Brasserie
Summary for restaurant:  Pietra Piccina
Summary for restaurant:  La Nonna


["The restaurant received mixed reviews. Some found the dishes bland and overpriced, and were disappointed with the overall experience. However, others praised the food and service, particularly the business lunch menu. The staff was generally courteous, but one reviewer felt ignored when they mentioned it was their girlfriend's birthday. Overall, opinions were divided and the rating seemed questionable.",
 'This restaurant is highly rated for its Chinese food, fresh ingredients, and friendly service. Customers appreciate the variety of dishes, quick service, and the authentic flavors. The owners are hospitable, and the overall experience is enjoyable. Limited seating and lack of a bathroom are minor drawbacks.',
 'Mixed reviews: Varied and delicious vegetarian options, juicy Beyond burger; wonderful French cuisine but sauces too salty; disappointed with café experience due to rude and stressed waiter; outstanding food and kitchen, but terrible and condescending service; cool place wit

#### Method 4:

In [31]:
prompt = "Please give me a summary of the reviews of this restaurant. I want one single summary for all 5 reviews. Please limit the summary to a maximum of 400 characters. If you don't, I might have to kill myself"

get_all_summaries(restaurants, prompt)

Summary for restaurant:  Wai's Kitchen
Summary for restaurant:  Sangeet
Summary for restaurant:  The Italian Shot Maxvorstadt
Summary for restaurant:  Conti Restaurant
Summary for restaurant:  Döner Öz Urfa


['Summary: This restaurant is highly praised for its fresh and flavorful Chinese food. The chicken dishes are particularly spectacular. The service is friendly and accommodating, with quick service and recommendations for dishes. The owners are hospitable and kind. Limited seating outside is available for enjoying a spring lunch. Overall, it is a favorite spot for Asian food in Munich.',
 'This restaurant has a cosy and warming atmosphere with excellent service and food quality. It offers a great Indian taste and has traditional decorations that add to its charm. The staff is friendly and the food is amazing. However, some reviewers pointed out that it is actually a Pakistani/Bangladeshi restaurant and not truly Indian. There were also complaints about rude staff and average food. Overall, it is a decent place for authentic North Indian food with a great ambiance and quick service.',
 'Review Summary: This Italian restaurant is highly praised for its excellent pizza and pasta dishes. T

## GPT4

In [18]:
def generate_summary_gpt4(client, reviews, prompt):
    #Concatenate all reviews
    review_text = "\n".join(review.get('text', '') for review in reviews)

    #Define the prompt for GPT
    prompt = f"{prompt} \n{review_text}"

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": prompt},
        ]
    )
    #Extract the generated summary from GPT's response
    summary = response.choices[0].message.content

    return summary

def get_summary_gpt4(place_id, prompt):
    # review_array = fetch_review(place_id)

    # if len(review_array) > 0:
    #     print("Found in database!")
    #     review = review_array[0]
    #     summary = review[1]
    # else:
    place_details = get_place_details(place_id)
    print("Summary for restaurant: ", place_details.get('name'))
    reviews = place_details.get('reviews', [])
    summary = generate_summary_gpt4(OpenAI(), reviews, prompt)
    # cursor.execute('INSERT INTO review_summaries (id, review_summary) VALUES (?, ?)', (place_id, summary))
    # conn.commit()

    return summary

def get_all_summaries_gpt4(restaurants, prompt):
    summaries = []
    for restaurant in restaurants:
        summaries.append(get_summary_gpt4(restaurant['place_id'], prompt))

    return summaries

In [19]:
prompt = "Please give me a summary of the reviews of this restaurant. I want one single summary for all 5 reviews. Please limit the summary to a maximum of 400 characters. It is extremely important that you limit the summary to a maximum of 400 characters."

get_all_summaries_gpt4(restaurants, prompt)

Summary for restaurant:  Conti Restaurant
Summary for restaurant:  Wai's Kitchen
Summary for restaurant:  Oskar Maria Brasserie
Summary for restaurant:  Pietra Piccina
Summary for restaurant:  La Nonna


['Reviews for the restaurant are mixed. Some patrons complain of overpricing, bland flavors, and potentially fake reviews lowering their overall ratings. However, other reviews celebrate the quality of food and service, with specific praise for the business lunch and mentioning that the staff was friendly and accommodating. There was disappointment in the lack of acknowledgment for a special occasion.',
 'The restaurant is highly popular for its Chinese food, with many praising its super fresh and flavorful ingredients, with no artificial flavors needed. The chicken dishes are particularly popular. The service is also praised as very friendly and hospitable. The restaurant offers timely service, serving food quickly. The vegetables are reported as crispand fresh, and the wontons perfect. Their food provides a feeling of a home-cooked meal. Some have noted limited outdoor seating and lack of an on-site bathroom.',
 'The restaurant offers a variety of vegetarian and vegan options, includ